<a href="https://colab.research.google.com/github/pranavvp16/End-to-End-Anime-Recomendation-Model/blob/main/Model_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_frame = pd.read_csv("drive/MyDrive/animeflv.csv")

In [ ]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3409 entries, 0 to 3408
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        3409 non-null   object 
 1   description  3189 non-null   object 
 2   year         3409 non-null   int64  
 3   type         3409 non-null   object 
 4   rate_start   3409 non-null   float64
 5   votes        3409 non-null   int64  
 6   status       3409 non-null   object 
 7   followers    3409 non-null   int64  
 8   episodes     3409 non-null   int64  
 9   genders      3403 non-null   object 
 10  url_anime    3409 non-null   object 
 11  image        3409 non-null   object 
dtypes: float64(1), int64(4), object(7)
memory usage: 319.7+ KB


In [ ]:
tans=data_frame['description'][0]

In [ ]:
data_frame.dropna(inplace=True)

In [ ]:
#data_frame['description'].head()

In [ ]:
pip install googletrans==3.1.0a0

In [ ]:
import googletrans
translator = googletrans.Translator()
df1 = data_frame['description'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))
#print(df1)

In [ ]:
data_frame["description"] = df1

In [ ]:
data_frame["genders"] = data_frame['genders'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))

In [ ]:
data_frame.rename(columns ={'genders':'genres'},inplace = True)

In [ ]:
#taking data for training set 
anime_df = data_frame[["title","description","genres","image"]]

In [ ]:
anime_df.head()

,title,description,genres,image
1,Dragon Ball Z Pelicula 03: La batalla más gran...,A band of deserters from Frieza's army led by ...,"Adventure,Science Fiction,Comedy,Fantasy,Shounen,",https://www3.animeflv.net/uploads/animes/cover...
2,Lodoss-tou Senki,The story centers around a young man named Par...,"Action,Adventure,Fantasy,Magic,Shounen,Superna...",https://www3.animeflv.net/uploads/animes/cover...
3,Fushigi no Umi no Nadia,The industrial revolution is over and the worl...,"Adventure,Science Fiction,Comedy,Historical,Ro...",https://www3.animeflv.net/uploads/animes/cover...
4,Dragon Ball Z Pelicula 02: El hombre más fuert...,"The situation begins with Dr. Uir?, who is rev...","Adventure,Science Fiction,Comedy,Fantasy,Shounen,",https://www3.animeflv.net/uploads/animes/cover...
5,Umezu Kazuo no Noroi,"Two separate stories. In the first titled ""Wha...","Terror,",https://www3.animeflv.net/uploads/animes/cover...


In [ ]:
anime_df.drop_duplicates(inplace = True)

In [ ]:
anime_df["genres"].value_counts()

Comedy, Slice of Life,                       52
Comedy,Schools,Slice of Life,                42
Comedy,                                      33
Music,                                       28
Action,Fantasy,                              19
                                             ..
Drama,Fantasy,Magic,Romance,Supernatural,     1
Science Fiction, Comedy, Ecchi, Shounen,      1
Mystery,Romance,Supernatural,Vampires,        1
Action,Adventure,Comedy,Ecchi,                1
Action, Fantasy, Shounen,                     1
Name: genres, Length: 1626, dtype: int64

In [ ]:
anime_df["image"].value_counts()

In [ ]:
#import urllib.request

#get_url= urllib.request.urlopen('https://www3.animeflv.net/uploads/animes/covers/3379.jpg')

#print("Response Status: "+ str(get_url.getcode()) )

In [ ]:
anime_df["description"] = [i.replace("'","")for i in anime_df["description"]]
anime_df["description"] = [i.replace(",","")for i in anime_df["description"]]
anime_df["description"] = [i.replace('-',"")for i in anime_df["description"]]
anime_df["title"] = [i.replace("Pelicula","Movie")for i in anime_df["title"]]

In [ ]:
anime_df["title"].head()

1    Dragon Ball Z Movie 03: La batalla más grande ...
2                                     Lodoss-tou Senki
3                              Fushigi no Umi no Nadia
4    Dragon Ball Z Movie 02: El hombre más fuerte d...
5                                 Umezu Kazuo no Noroi
Name: title, dtype: object

In [ ]:
anime_df["description"]=anime_df["description"].apply(lambda x:x.split())
anime_df["genres"]=anime_df["genres"].apply(lambda x:x.split())

In [ ]:
anime_df['description'].head()

0    [A, band, of, deserters, from, Friezas, army, ...
1    [The, story, centers, around, a, young, man, n...
2    [The, industrial, revolution, is, over, and, t...
3    [The, situation, begins, with, Dr., Uir?, who,...
4    [Two, separate, stories., In, the, first, titl...
Name: description, dtype: object

In [ ]:
anime_df['genres'].head()

0    [Adventure,Science, Fiction,Comedy,Fantasy,Sho...
1    [Action,Adventure,Fantasy,Magic,Shounen,Supern...
2    [Adventure,Science, Fiction,Comedy,Historical,...
3    [Adventure,Science, Fiction,Comedy,Fantasy,Sho...
4                                            [Terror,]
Name: genres, dtype: object

In [ ]:
#anime_df['title'] = data_frame['title']

In [ ]:
anime_df["tags"] = anime_df['description'] + anime_df['genres']

In [ ]:
anime_df.isna().sum()

title          0
description    0
genres         0
image          0
tags           0
dtype: int64

In [ ]:
#multi_chars = "[']"
#def convert(lst , sym): 
 #for i in lst:
  #temp_string = i
  #for ch in sym:
    #temp_string = temp_string.replace(ch,"")
    #for j in temp_string:
      #dta = []
      #dta.append(j)
 #return dta 

In [ ]:
anime_df["tags"]=anime_df["tags"].apply(lambda x:[i.replace(" ","")for i in x])
anime_df["tags"] = anime_df["tags"].apply(lambda x:" ".join(x))

In [ ]:
anime_df.head()

,title,description,genres,image,tags
0,Dragon Ball Z Movie 03: La batalla más grande ...,"[A, band, of, deserters, from, Friezas, army, ...","[Adventure,Science, Fiction,Comedy,Fantasy,Sho...",https://www3.animeflv.net/uploads/animes/cover...,A band of deserters from Friezas army led by t...
1,Lodoss-tou Senki,"[The, story, centers, around, a, young, man, n...","[Action,Adventure,Fantasy,Magic,Shounen,Supern...",https://www3.animeflv.net/uploads/animes/cover...,The story centers around a young man named Par...
2,Fushigi no Umi no Nadia,"[The, industrial, revolution, is, over, and, t...","[Adventure,Science, Fiction,Comedy,Historical,...",https://www3.animeflv.net/uploads/animes/cover...,The industrial revolution is over and the worl...
3,Dragon Ball Z Movie 02: El hombre más fuerte d...,"[The, situation, begins, with, Dr., Uir?, who,...","[Adventure,Science, Fiction,Comedy,Fantasy,Sho...",https://www3.animeflv.net/uploads/animes/cover...,The situation begins with Dr. Uir? who is revi...
4,Umezu Kazuo no Noroi,"[Two, separate, stories., In, the, first, titl...","[Terror,]",https://www3.animeflv.net/uploads/animes/cover...,"Two separate stories. In the first titled ""Wha..."


In [ ]:
anime_df=anime_df.reset_index(drop=True)

In [ ]:
animef_df = anime_df[["title","tags"]]

In [ ]:
animef_df.head()

,title,tags
0,Dragon Ball Z Movie 03: La batalla más grande ...,A band of deserters from Friezas army led by t...
1,Lodoss-tou Senki,The story centers around a young man named Par...
2,Fushigi no Umi no Nadia,The industrial revolution is over and the worl...
3,Dragon Ball Z Movie 02: El hombre más fuerte d...,The situation begins with Dr. Uir? who is revi...
4,Umezu Kazuo no Noroi,"Two separate stories. In the first titled ""Wha..."


In [ ]:
animef_df["tags"] = anime_df["tags"].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=4000,stop_words="english")
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()

In [ ]:
ps.stem("abilities")

'abil'

In [ ]:
def stem(text):
  y=[]

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y) 

In [ ]:
animef_df["tags"][1]

'the story centers around a young man named parn the son of a disgraced knight. part of this motivation to venture out is to find out what happened to his father and restore his familys honor. despite his inexperience parn is considered the leader who is accompanied by his best friend a forest boy named etoh his friend and sometimes advisor slayn (and later slayns lover leylia) and his new mentor ghim. . they are joined by parns romantic interest the high elf deedlit who comes from the forest of no return seeking a solution to her peoples isolationism and an end to extinction. throughout the series parn comes into contact with his friends and his enemies alike. his allies including king kashue fahn shiris and orson; his enemies including emperor beld ashram and the evil ghoul wagnard. action,adventure,fantasy,magic,shounen,supernatural,'

In [ ]:
stem(animef_df["tags"][1])

'the stori center around a young man name parn the son of a disgrac knight. part of thi motiv to ventur out is to find out what happen to hi father and restor hi famili honor. despit hi inexperi parn is consid the leader who is accompani by hi best friend a forest boy name etoh hi friend and sometim advisor slayn (and later slayn lover leylia) and hi new mentor ghim. . they are join by parn romant interest the high elf deedlit who come from the forest of no return seek a solut to her peopl isolation and an end to extinction. throughout the seri parn come into contact with hi friend and hi enemi alike. hi alli includ king kashu fahn shiri and orson; hi enemi includ emperor beld ashram and the evil ghoul wagnard. action,adventure,fantasy,magic,shounen,supernatural,'

In [ ]:
animef_df["tags"]=animef_df["tags"].apply(stem)

In [ ]:
vectors = cv.fit_transform(animef_df["tags"]).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors).shape

(3182, 3182)

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

array([1.        , 0.04352858, 0.11795356, ..., 0.07878386, 0.02519763,
       0.14342743])

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

In [ ]:
def recommend(anime):
  anime_index = animef_df[animef_df['title']==anime].index[0]
  distances = similarity[anime_index]
  anime_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in anime_list:
    print(animef_df.iloc[i[0]].title)

In [ ]:
recommend('Fushigi no Umi no Nadia')

Houkago Saikoro Club
Mai-HiME
Ima, Soko ni Iru Boku
Hi-sCoool! SeHa Girl
Maho Shojo Madoka Magica


In [ ]:
import pickle 
pickle.dump(animef_df.to_dict(),open('anime_dict.pkl','wb'))
pickle.dump(animef_df,open('anime.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
pickle.dump(image_urls,open('image_urls.pkl','wb'))

In [ ]:
image_urls[0]